# 메타데이터에 모델 점수를 연결

In [1]:
import pandas as pd
import pickle
import numpy as np
from pathlib import Path

norm_path = Path('./experiment/matches_scores/normal')
norm_list = list(norm_path.glob('*'))

zero_path = Path('./experiment/matches_scores/zero_h0')
zero_list = list(zero_path.glob('*'))

mlp_path = Path('./experiment/matches_scores/mlp')
mlp_list = list(mlp_path.glob('*'))

with open('./Dataset/meta_analysis.ftr', 'rb') as file:
    meta = pd.read_feather(file)

meta['score_norm'] = np.nan
for idx, norm_file in enumerate(norm_list):
    print(f'[Norm] ({idx:5d}/{len(norm_list)}) {idx/len(norm_list)*100:3.1f}% progressing...', end='\r')
    mat_name = norm_file.name[:-4]

    with norm_file.open('rb') as file:
        scores = pickle.load(file)
    scores = [scores[idx].sum().item() for idx in range(10)]

    meta.loc[(meta.mat_no==mat_name), 'score_norm'] = scores

meta['score_zero'] = np.nan
for idx, zero_file in enumerate(zero_list):
    print(f'[Zero] ({idx:5d}/{len(zero_list)}) {idx/len(zero_list)*100:3.1f}% progressing...', end='\r')
    mat_name = zero_file.name[:-4]

    with zero_file.open('rb') as file:
        scores = pickle.load(file)
    scores = [scores[idx].sum().item() for idx in range(10)]

    meta.loc[(meta.mat_no==mat_name), 'score_zero'] = scores

meta['score_mlp'] = np.nan
for idx, mlp_file in enumerate(mlp_list):
    print(f'[MLP] ({idx:5d}/{len(mlp_list)}) {idx/len(mlp_list)*100:3.1f}% progressing...', end='\r')
    mat_name = mlp_file.name[:-4]

    with mlp_file.open('rb') as file:
        scores = pickle.load(file)
    scores = [scores[idx].sum().item() for idx in range(10)]

    meta.loc[(meta.mat_no==mat_name), 'score_mlp'] = scores

meta.to_feather('./Dataset/meta_analysis_with_score.ftr')
meta

,mat_no,player,puuid,lane,kda,gold_earned,gold_spent,creep,teamid,win,score_norm,score_zero,score_mlp
0,210513_na1_3902040674,0,dQBetmuFhTOgyCJl0t444hFACPw3iikFifzfWb3OPo9Iqn...,TOP,0.000000,4153,3800,55,100,False,-0.043986,0.072929,0.315355
1,210513_na1_3902040674,1,2wouOlN1UmGWq_QLRLJp3TFhOpt8lCJlZg2Ic3MCp_pbVz...,JUNGLE,1.250000,5876,5883,29,100,False,-0.116598,0.119087,0.339367
2,210513_na1_3902040674,2,aFe7h8n6sb_sSns7RRyQH-NNc4mDpI8x3vCqnCd-tsMWoP...,MIDDLE,0.428571,5403,3500,79,100,False,0.000979,0.081865,0.323561
3,210513_na1_3902040674,3,dyHw7mRJED76t1yXruTaoq6xCgW7islnAlKPCVwRotWsDQ...,BOTTOM,0.600000,7713,6200,91,100,False,-0.057029,0.107460,0.404525
4,210513_na1_3902040674,4,7PUdBdt_fgbFxQe_V2t2b_In8UA9xcKJEJPEgs5Trz-1LA...,UTILITY,0.500000,5722,4900,28,100,False,-0.083076,0.109865,0.376910
...,...,...,...,...,...,...,...,...,...,...,...,...,...
455745,210517_jp1_302513885,5,28_Nasidr_s-ReqE-t2HOfL5dgeouNRHDd8pAFolVdHyo7...,TOP,7.000000,7216,6925,106,200,True,0.148615,0.118142,0.479662
455746,210517_jp1_302513885,6,2zbLfaa53OCAEdOSmS6_nlenqMAa7ShIJIhp-6aSrr-G5y...,JUNGLE,12.000000,6964,6875,8,200,True,0.102792,0.175900,0.554105
455747,210517_jp1_302513885,7,E44uriXb-ym4F7UjUfeoZEXHFwrZS61Cs9JpB-dVwylFEG...,MIDDLE,6.000000,7953,7160,124,200,True,0.179705,0.143393,0.450013
455748,210517_jp1_302513885,8,Jkc5sKy3LYx-fB5-C7gG6g8bbXjtwdw2gV_0PFOlcXapZ9...,BOTTOM,2.000000,6862,5925,120,200,True,0.151155,0.142550,0.339654


# 각 매치별 플레이어 KDA, Gold, Creep score, 모델 점수를 순위화한 데이터 생성

In [1]:
import pandas as pd
import pickle
import numpy as np
from tqdm import tqdm

with open('./Dataset/meta_analysis_with_score.ftr', 'rb') as file:
    meta = pd.read_feather(file)
meta.rename(columns={'gold_earned':'gold', 'score_norm':'score_model1', 'score_zero':'score_model2', 'score_mlp':'score_model3'}, inplace=True)
columns = ['match', 'player', 'kda', 'gold', 'creep', 'score_model1', 'score_model2', 'score_model3']

block_num = int(len(meta) / 10)
ar = np.zeros([len(meta), len(columns)], dtype=np.int8)
df = pd.DataFrame(ar, columns=columns)

for idx in tqdm(range(block_num)):
    sdx = idx * 10
    edx = sdx + 9
    tmp = meta.loc[sdx:edx].copy()
    df.loc[sdx:edx, 'match'] = tmp['mat_no']
    df.loc[sdx:edx, 'player'] = tmp['player']

    for col in columns[2:]:
        tmp.sort_values(by=col, axis=0, inplace=True, ignore_index=True, ascending=False)
        data = [int(tmp.index[tmp.player==pi].item()) for pi in range(10)]
        df.loc[sdx:edx, col] = data

tqdm.pandas()
df['avg'] = df.progress_apply((lambda x: int((x.gold+x.creep+x.kda)/3)), axis=1)
df['avg_nocreep'] = df.progress_apply((lambda x: int((x.gold+x.kda)/2)), axis=1)

df.to_feather('./experiment/analysis_result4.ftr')
df

100%|██████████| 455750/455750 [00:03<00:00, 120741.76it/s]


,match,player,kda,gold,creep,score_model1,score_model2,score_model3,avg,avg_nocreep
0,210513_na1_3902040674,0,9,9,5,6,9,9,7,9
1,210513_na1_3902040674,1,5,6,6,9,5,7,5,5
2,210513_na1_3902040674,2,8,8,4,5,8,8,6,8
3,210513_na1_3902040674,3,6,4,3,7,7,5,4,5
4,210513_na1_3902040674,4,7,7,7,8,6,6,7,7
...,...,...,...,...,...,...,...,...,...,...
455745,210517_jp1_302513885,5,1,1,4,2,6,1,2,1
455746,210517_jp1_302513885,6,0,2,9,4,0,0,3,1
455747,210517_jp1_302513885,7,2,0,0,0,1,2,0,1
455748,210517_jp1_302513885,8,5,3,2,1,2,5,3,4
